In [139]:
import GridCalEngine.api as gce
import numpy as np

In [140]:
def p(m, a):
    return abs(m) * np.exp(1j * np.deg2rad(a))

In [141]:
# %% Cria o objeto Rede
# declare a circuit object
grid = gce.MultiCircuit()

In [142]:
# %% Cria a barra 1g e um gerador. Conecta o gerador na barra criada

bus1 = gce.Bus('Bus 1', Vnom=13.8)
bus1.is_slack = True  # we may mark the bus a slack
grid.add_bus(bus1)

gen1 = gce.Generator('Slack Generator',
                     vset=1.05,
                     Snom=100.0,
                     is_controlled=True,
                     power_factor=0.95,
                     x1=0.15, x2=0.17, x0=0.05)
grid.add_generator(bus1, gen1)

.::65680d6678364980a850e1865e866012::Slack Generator

In [143]:
# %% Cria a barra 2 e um transformador. Conecta o transformador entre as barras 1 e 2 
bus2 = gce.Bus('Bus 2', Vnom=138.0)
grid.add_bus(bus2)

# add Transformer connecting the buses 1 and 2
trafo1 = gce.Transformer2W(bus_from=bus2,
                           bus_to=bus1,
                           name='Transformer 1',
                           HV=138.0,
                           LV=13.8,
                           rate=100.0,
                           x=0.1,
                           x2=0.1,
                           x0=0.1,
                           conn=gce.WindingsConnection.GD)
grid.add_transformer2w(trafo1)

In [144]:
n = 0.5

# %% Cria a barra 3 uma LT. Conecta a LT entre as barras 2 e 3
bus3 = gce.Bus('Bus 3', Vnom=138.0)
grid.add_bus(bus3)

# %% Cria a barra 3 uma LT. Conecta a LT entre as barras 2 e 3
bus23 = gce.Bus('Bus 23', Vnom=138.0)
grid.add_bus(bus23)

# add Line connecting the buses
line1 = gce.Line(bus_from=bus2,
                 bus_to=bus23,
                 name='line 1',
                 x=0.1050 * n,
                 x2=0.1050 * n,
                 x0=0.315 * n,
                 rate=100)
grid.add_line(line1)

line2 = gce.Line(bus_from=bus23,
                 bus_to=bus3,
                 name='line 2',
                 x=0.1050 * (1.0 - n),
                 x2=0.1050 * (1.0 - n),
                 x0=0.315 * (1.0 - n),
                 rate=100)
grid.add_line(line2)

.::7af917c81c60403e91cfc717320f87c4::line 2

In [145]:
# %% Cria a barra 4 e um transformador. Conecta o transformador entre as barras 3 e 4
bus4 = gce.Bus('Bus 4', Vnom=13.8)
grid.add_bus(bus4)

# add Transformer connecting the buses 1 and 2
trafo2 = gce.Transformer2W(bus_from=bus3,
                           bus_to=bus4,
                           name='Transformer 2',
                           HV=138.0,
                           LV=13.8,
                           rate=100.0,
                           x=0.1,
                           x2=0.1,
                           x0=0.1,
                           conn=gce.WindingsConnection.GD)
grid.add_transformer2w(trafo2)

In [146]:
# %% Cria a carga (gerador/motor síncrono) e conecta a carga na barra 4m

motor1 = gce.Generator('Sincronous Motor',
                       P=-100.0,
                       #P=0.0,
                       #is_controlled=True,
                       is_controlled=False,
                       x1=0.20,
                       x2=0.21,
                       x0=0.10 + 3.0 * 0.05,
                       power_factor=0.95,
                       vset=1.05)

grid.add_generator(bus4, motor1)


.::e4f0745992e54c94829dac40ace32596::Sincronous Motor

In [147]:
# %% Executa cálculo de fluxo de carga
pf_options = gce.PowerFlowOptions()
pf = gce.PowerFlowDriver(grid, pf_options)
pf.run()

In [148]:
pf.results.get_bus_df()

,Vm,Va,P,Q
Bus 1,1.0500,0.0000,100.0000,76.8868
Bus 2,0.9814,-5.5689,-0.0000,0.0000
Bus 3,0.9208,-12.2411,-0.0000,0.0000
Bus 23,0.9495,-8.7986,-0.0000,0.0000
Bus 4,0.8762,-19.3607,-100.0000,-32.8684


In [149]:
pf.results.get_branch_df()

,Pf,Qf,Pt,Qt,loading
line 1,100.0000,62.4546,-100.0000,-54.8776,100.0000
line 2,100.0000,54.8776,-100.0000,-47.3007,100.0000
Transformer 1,-100.0000,-62.4546,100.0000,76.8868,-100.0000
Transformer 2,100.0000,47.3007,-100.0000,-32.8684,100.0000


## 3ph Short Circuit

In [150]:
# %% Executa cálculo de curto-circuito
fault_index_1 = 3

sc1_options = gce.ShortCircuitOptions(bus_index=fault_index_1, 
                                     fault_type=gce.FaultType.ph3)

sc1 = gce.ShortCircuitDriver(grid,
                            options=sc1_options, 
                            pf_options=pf_options, 
                            pf_results=pf.results)
sc1.run()

### Positive Sequence Currents

In [151]:
pf.results.bus_names

array(['Bus 1', 'Bus 2', 'Bus 3', 'Bus 23', 'Bus 4'], dtype=object)

In [152]:
pf.results.branch_names

array(['line 1', 'line 2', 'Transformer 1', 'Transformer 2'], dtype=object)

In [153]:
for i, j in zip(sc1.results.branch_names, sc1.results.It1):
    print(f"Branch name: {i:15} | It Module: {abs(j):.4f} | {np.angle(j, deg=True):.4f}")

print("\n----------------------------------------")

for i, j in zip(sc1.results.branch_names, sc1.results.If1):
    print(f"Branch name: {i:15} | If Module: {abs(j):.4f} | {np.angle(j, deg=True):.4f}")

Branch name: line 1          | It Module: 3.8632 | 97.0217
Branch name: line 2          | It Module: 2.3633 | -125.2627
Branch name: Transformer 1   | It Module: 3.8632 | -82.9783
Branch name: Transformer 2   | It Module: 2.3633 | -125.2627

----------------------------------------
Branch name: line 1          | If Module: 3.8632 | -82.9783
Branch name: line 2          | If Module: 2.3633 | 54.7373
Branch name: Transformer 1   | If Module: 3.8632 | 97.0217
Branch name: Transformer 2   | If Module: 2.3633 | 54.7373


In [154]:
# cálculo da corrente de curto circuito na barra sob falta (barra = 3, index = 2)
Icc3ph_gc = sc1.results.If1[0] + sc1.results.It1[1]
print(f"Corrente de cc 3ph: {abs(Icc3ph_gc):.4f} | {np.angle(Icc3ph_gc, deg=True):.4f}")

Corrente de cc 3ph: 5.8325 | -98.7986


## Assymetric Shor Circuit

In [155]:
# %% Executa cálculo de curto-circuito
# fault_index_2 = 1
fault_index_2 = 3
sc2_options = gce.ShortCircuitOptions(bus_index=fault_index_2, 
                                      fault_type=gce.FaultType.LG)

sc2 = gce.ShortCircuitDriver(grid,
                             options=sc2_options,
                             pf_options=pf_options,
                             pf_results=pf.results)
sc2.run()

### Positive Sequence Currents

In [156]:
for i, j in zip(sc2.results.branch_names, sc2.results.It1):
    print(f"Branch name: {i:15} | Fault Current Value: {abs(j):.4f} | {np.angle(j, deg=True):.4f}")

Branch name: line 1          | Fault Current Value: 1.9858 | 143.2311
Branch name: line 2          | Fault Current Value: 1.1165 | -139.4044
Branch name: Transformer 1   | Fault Current Value: 1.9858 | -66.7689
Branch name: Transformer 2   | Fault Current Value: 1.1165 | -169.4044


### Negative Sequence Currents

In [157]:
for i, j in zip(sc2.results.branch_names, sc2.results.It2):
    print(f"Branch name: {i:15} | Fault Current Value: {abs(j):.4f} | {np.angle(j, deg=True):.4f}")

Branch name: line 1          | Fault Current Value: 1.0871 | 111.2014
Branch name: line 2          | Fault Current Value: 0.9672 | -68.7986
Branch name: Transformer 1   | Fault Current Value: 1.0871 | -38.7986
Branch name: Transformer 2   | Fault Current Value: 0.9672 | -38.7986


### Zero Sequence Currents

In [158]:
for i, j in zip(sc2.results.branch_names, sc2.results.It0):
    print(f"Branch name: {i:15} | Fault Current Value: {abs(j):.4f} | {np.angle(j, deg=True):.4f}")

Branch name: line 1          | Fault Current Value: 1.0271 | 111.2014
Branch name: line 2          | Fault Current Value: 1.0271 | -68.7986
Branch name: Transformer 1   | Fault Current Value: 0.0000 | 0.0000
Branch name: Transformer 2   | Fault Current Value: 0.0000 | 0.0000


In [159]:
grid.buses

[.::baaf7a13963d42f5ba881285fc6fe392::Bus 1,
 .::78e909573463476fb8e2d03ad6a18907::Bus 2,
 .::e8e76eacc3ab4690a155769e7b0feaec::Bus 3,
 .::cc15356ac028423d96c452b5c1ddd5a5::Bus 23,
 .::144c0a6765124971a81597bf36844842::Bus 4]

In [160]:
sc2.results.If1

array([ 1.5907287 -1.18867172j,  0.84781041+0.72654855j, -1.59072872+1.18867171j,  0.8478103 +0.72654849j])

In [161]:
sc2.results.It1

array([-1.5907287 +1.18867172j, -0.84781041-0.72654855j,  0.78327563-1.82478426j, -1.0974995 -0.2053043j ])

### Cálculo da correntes de Curto-Circuito na barra sob falta

In [162]:
sc2.results.branch_names

array(['line 1', 'line 2', 'Transformer 1', 'Transformer 2'], dtype=object)

In [163]:
# cálculo da corrente de curto circuito na barra sob falta
abs(sc2.results.If1[0] + sc2.results.It1[1])

2.054262958302996

In [164]:
abs(sc2.results.If2[0] + sc2.results.It2[1])

2.05426296286584

In [165]:
abs(sc2.results.If0[0] + sc2.results.It0[1])

2.054262962865841

In [166]:
def p(m, a):
    return abs(m) * (np.cos(np.deg2rad(a)) + 1j * np.sin(np.deg2rad(a)))

In [167]:
a = p(1.0, 120.0)
A = np.array([
             [1.0, 1.0, 1.0],
             [1.0, a**2, a],
             [1.0, a, a**2]])
A.shape = (3, 3)

In [168]:
np.linalg.inv(A) * 3

array([[ 1. -0.j       ,  1. +0.j       ,  1. +0.j       ],
       [ 1. +0.j       , -0.5+0.8660254j, -0.5-0.8660254j],
       [ 1. +0.j       , -0.5-0.8660254j, -0.5+0.8660254j]])

In [169]:
Y1 = np.array([
    [1.0 / 0.15j + 1.0 / 0.1j, -1.0/0.1j, 0.0, 0.0, 0.0], # Bus 1 ok
    [-1.0/0.1j, 1.0 / 0.1j + 1.0 / (0.105j * n), 0.0, 0.0, -1.0 / (0.105j * n)], # Bus 2 ok
    [0.0, 0.0, 1.0 / 0.1j + 1.0 / (0.105j * (1.0 - n)), -1.0/0.1j, -1.0 / (0.105j * (1.0 - n))], # Bus 3 ok
    [0.0, 0.0, -1.0/0.1j, 1.0 / 0.1j + 1.0 / 0.2j, 0.0], # Bus 4 ok
    [0.0,  -1.0 / (0.105j * n),  -1.0 / (0.105j * (1.0 - n)), 0.0, 1.0 / (0.105j * n) + 1.0 / (0.105j * (1.0 - n))] # Bus 23 ok
    ]) 
Y1

array([[0.-16.66666667j, 0.+10.j        , 0. +0.j        , 0. +0.j        , 0. +0.j        ],
       [0.+10.j        , 0.-29.04761905j, 0. +0.j        , 0. +0.j        , 0.+19.04761905j],
       [0. +0.j        , 0. +0.j        , 0.-29.04761905j, 0.+10.j        , 0.+19.04761905j],
       [0. +0.j        , 0. +0.j        , 0.+10.j        , 0.-15.j        , 0. +0.j        ],
       [0. +0.j        , 0.+19.04761905j, 0.+19.04761905j, 0. +0.j        , 0.-38.0952381j ]])

In [170]:
Z1 = np.linalg.inv(Y1)
Z1

array([[0.+0.11564885j, 0.+0.09274809j, 0.+0.06870229j, 0.+0.04580153j, 0.+0.08072519j],
       [0.+0.09274809j, 0.+0.15458015j, 0.+0.11450382j, 0.+0.07633588j, 0.+0.13454198j],
       [0.+0.06870229j, 0.+0.11450382j, 0.+0.16259542j, 0.+0.10839695j, 0.+0.13854962j],
       [0.+0.04580153j, 0.+0.07633588j, 0.+0.10839695j, 0.+0.1389313j , 0.+0.09236641j],
       [0.+0.08072519j, 0.+0.13454198j, 0.+0.13854962j, 0.+0.09236641j, 0.+0.1627958j ]])

In [171]:
Y2 = np.array([
    [1.0 / 0.17j + 1.0 / 0.1j, -1.0/0.1j, 0.0, 0.0, 0.0], # Bus 1 ok
    [-1.0/0.1j, 1.0 / 0.1j + 1.0 / (0.105j * n), 0.0, 0.0, -1.0 / (0.105j * n)], # Bus 2 ok
    [0.0, 0.0, 1.0 / 0.1j + 1.0 / (0.105j * (1.0 - n)), -1.0/0.1j, -1.0 / (0.105j * (1.0 - n))], # Bus 3 ok
    [0.0, 0.0, -1.0/0.1j, 1.0 / 0.1j + 1.0 / 0.21j, 0.0], # Bus 4 ok
    [0.0,  -1.0 / (0.105j * n),  -1.0 / (0.105j * (1.0 - n)), 0.0, 1.0 / (0.105j * n) + 1.0 / (0.105j * (1.0 - n))] # Bus 23 ok
    ]) 
Y2

array([[0.-15.88235294j, 0.+10.j        , 0. +0.j        , 0. +0.j        , 0. +0.j        ],
       [0.+10.j        , 0.-29.04761905j, 0. +0.j        , 0. +0.j        , 0.+19.04761905j],
       [0. +0.j        , 0. +0.j        , 0.-29.04761905j, 0.+10.j        , 0.+19.04761905j],
       [0. +0.j        , 0. +0.j        , 0.+10.j        , 0.-14.76190476j, 0. +0.j        ],
       [0. +0.j        , 0.+19.04761905j, 0.+19.04761905j, 0. +0.j        , 0.-38.0952381j ]])

In [172]:
Z2 = np.linalg.inv(Y2)
Z2

array([[0.+0.12781022j, 0.+0.1029927j , 0.+0.07693431j, 0.+0.05211679j, 0.+0.0899635j ],
       [0.+0.1029927j , 0.+0.16357664j, 0.+0.12218978j, 0.+0.08277372j, 0.+0.14288321j],
       [0.+0.07693431j, 0.+0.12218978j, 0.+0.16970803j, 0.+0.1149635j , 0.+0.14594891j],
       [0.+0.05211679j, 0.+0.08277372j, 0.+0.1149635j , 0.+0.14562044j, 0.+0.09886861j],
       [0.+0.0899635j , 0.+0.14288321j, 0.+0.14594891j, 0.+0.09886861j, 0.+0.17066606j]])

In [173]:
Y0 = np.array([[1.0 / (0.05j), 0.0, 0.0, 0.0, 0.0], # Bus 1 ok
               [0.0, 1.0 / 0.1j + 1.0 / (0.315j * n), 0.0, 0.0, -1.0 / (0.315j * n)], # Bus 2 ok
               [0.0, 0.0, 1.0 / 0.1j + 1.0 / (0.315j * (1.0 - n)), 0.0, -1.0 / (0.315j * (1.0 - n))], # Bus 3 ok
               [0.0, 0.0, 0.0, 1.0 / (0.1j + 0.15j), 0.0], # Bus 4 ok
               [0.0, -1.0 / (0.315j * n), -1.0 / (0.315j * (1.0 - n)), 0.0, 1.0 / (0.315j * n) + 1.0 / (0.315j * (1.0 - n))] # Bus 23 ok
               ])
Y0

array([[0.-20.j        , 0. +0.j        , 0. +0.j        , 0. +0.j        , 0. +0.j        ],
       [0. +0.j        , 0.-16.34920635j, 0. +0.j        , 0. +0.j        , 0. +6.34920635j],
       [0. +0.j        , 0. +0.j        , 0.-16.34920635j, 0. +0.j        , 0. +6.34920635j],
       [0. +0.j        , 0. +0.j        , 0. +0.j        , 0. -4.j        , 0. +0.j        ],
       [0. +0.j        , 0. +6.34920635j, 0. +6.34920635j, 0. +0.j        , 0.-12.6984127j ]])

In [174]:
Z0 = np.linalg.inv(Y0)
Z0

array([[ 0.+0.05j      ,  0.+0.j        ,  0.+0.j        ,  0.+0.j        ,  0.+0.j        ],
       [ 0.+0.j        ,  0.+0.08058252j,  0.+0.01941748j,  0.+0.j        ,  0.+0.05j      ],
       [ 0.+0.j        ,  0.+0.01941748j,  0.+0.08058252j, -0.+0.j        ,  0.+0.05j      ],
       [-0.+0.j        , -0.+0.j        , -0.+0.j        ,  0.+0.25j      , -0.+0.j        ],
       [ 0.+0.j        ,  0.+0.05j      ,  0.+0.05j      ,  0.+0.j        ,  0.+0.12875j   ]])

In [175]:
# cálculo do curto-circuito 3ph na barra 23
Icc3ph = 1.05 / Z1[4, 4]
Icc3ph

-6.449797784420603j

In [176]:
# Cálculo da corrente de falta monofásica na barra 23:
I0 = 1.05 / (Z1[4, 4] + Z2[4, 4] + Z0[4, 4])
I0

-2.2716855430315865j

In [177]:
Ia = 3 * I0
Ia

-6.81505662909476j

In [178]:
# Cálculo da corrente de falta bifásica na barra 23:
I1 = 1.05 / (Z1[4, 4] + Z2[4, 4])
I1

-3.14878589188322j

In [179]:
I2 = -I1

In [180]:
abs(np.dot(A,np.array([[0.0, I1, I2]]).transpose()))

array([[0.        ],
       [5.45385715],
       [5.45385715]])

In [181]:
# Cálculo da falta bifásica à terra na barra 23:
I1 = 1.05 / (Z1[4, 4] + (Z2[4, 4] * Z0[4, 4]) / (Z2[4, 4] + Z0[4, 4]))
I1

-4.445708423427752j

In [182]:
I2 = -I1 * Z0[4, 4] / (Z0[4, 4] + Z2[4, 4])
I2

1.9116708789306744j

In [183]:
I0 = -I1 * Z2[4, 4] / (Z0[4, 4] + Z2[4, 4])
I0

2.5340375444970777j

In [184]:
abs(np.dot(A,np.array([[I0, I1, I2]]).transpose()))

array([[0.        ],
       [6.69030887],
       [6.69030887]])